In [1]:
import os

# Change the current working directory
new_directory = r"C:\Users\Pranav Darekar\Documents"
os.chdir(new_directory)

# Print the current working directory to confirm the change
print("Current Working Directory:", os.getcwd())


Current Working Directory: C:\Users\Pranav Darekar\Documents


In [2]:
import pandas as pd

# Load the CSV file using the full path
file_path_1 = r"C:\Users\Pranav Darekar\Documents\all_coins_ohlcv_filtered"
df_ohlcv = pd.read_csv(file_path_1)

file_path_2 = r"C:\Users\Pranav Darekar\Documents\crypto_listings_latest_sorted"
df_crypto_listings_sorted = pd.read_csv(file_path_2)


In [3]:
# Set the slug column as the index for both DataFrames
df_ohlcv.set_index('symbol', inplace=True)
df_crypto_listings_sorted.set_index('symbol', inplace=True)

# Perform an inner join on cmc_rank
df = df_crypto_listings_sorted[['cmc_rank']].join(df_ohlcv, how='inner')

# Filter rows where 'cmc_rank' is between 1 and 100 inclusive
df = df[(df['cmc_rank'] >= 1) & (df['cmc_rank'] <= 1001)]




In [4]:
# Ensure the timestamp column is in datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Sort the DataFrame by 'slug' and 'timestamp' columns
df.sort_values(by=['slug', 'timestamp'], inplace=True)

# Perform time-series calculations within each group (each cryptocurrency)
grouped = df.groupby('slug')




In [5]:
# Calculate percentage change for each cryptocurrency
df['m_pct_1d'] = grouped['close'].pct_change()

In [6]:
# Calculate cumulative returns for each cryptocurrency
df['d_pct_cum_ret'] = (1 + df['m_pct_1d']).groupby(df['slug']).cumprod() - 1

In [7]:
"""import pandas as pd

# Sample DataFrame creation for demonstration
# df = pd.read_csv('your_file.csv')  # Uncomment this line to load your actual DataFrame

# Ensure 'timestamp' is in datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Extract the year from the timestamp and add it as a new column
df['year'] = df['timestamp'].dt.year

# Calculate the number of years between the first and last timestamp for each cryptocurrency
df['years'] = df.groupby('slug')['timestamp'].transform(lambda x: (x.max() - x.min()).days / 365.25)

# Get the first and last value of 'd_pct_cum_ret' for CAGR calculation, grouped by both 'slug' and 'year'
df['beginning_value'] = df.groupby(['slug', 'year'])['d_pct_cum_ret'].transform('first')
df['ending_value'] = df.groupby(['slug', 'year'])['d_pct_cum_ret'].transform('last')

# Calculate CAGR for each combination of 'slug' and 'year'
df['CAGR'] = (df['ending_value'] / df['beginning_value']) ** (1 / df['years']) - 1


# Drop intermediate columns if needed
df = df.drop(columns=['beginning_value', 'ending_value'])

# Display the DataFrame
print(df)"""


"import pandas as pd\n\n# Sample DataFrame creation for demonstration\n# df = pd.read_csv('your_file.csv')  # Uncomment this line to load your actual DataFrame\n\n# Ensure 'timestamp' is in datetime format\ndf['timestamp'] = pd.to_datetime(df['timestamp'])\n\n# Extract the year from the timestamp and add it as a new column\ndf['year'] = df['timestamp'].dt.year\n\n# Calculate the number of years between the first and last timestamp for each cryptocurrency\ndf['years'] = df.groupby('slug')['timestamp'].transform(lambda x: (x.max() - x.min()).days / 365.25)\n\n# Get the first and last value of 'd_pct_cum_ret' for CAGR calculation, grouped by both 'slug' and 'year'\ndf['beginning_value'] = df.groupby(['slug', 'year'])['d_pct_cum_ret'].transform('first')\ndf['ending_value'] = df.groupby(['slug', 'year'])['d_pct_cum_ret'].transform('last')\n\n# Calculate CAGR for each combination of 'slug' and 'year'\ndf['CAGR'] = (df['ending_value'] / df['beginning_value']) ** (1 / df['years']) - 1\n\n\n# D

In [8]:
import pandas as pd

# Define the confidence level, e.g., 95%
confidence_level = 0.95

# Calculate Historical VaR for each cryptocurrency
VaR_df = df.groupby('slug').apply(lambda x: x['m_pct_1d'].quantile(1 - confidence_level))
VaR_df = VaR_df.reset_index(name='d_pct_var')

# Calculate CVaR for each cryptocurrency
CVaR_df = df.groupby('slug').apply(lambda x: x['m_pct_1d'][x['m_pct_1d'] <= x['m_pct_1d'].quantile(1 - confidence_level)].mean())
CVaR_df = CVaR_df.reset_index(name='d_pct_cvar')

# Merge VaR and CVaR back into the original DataFrame
df = df.merge(VaR_df, on='slug', how='left')
df = df.merge(CVaR_df, on='slug', how='left')

print(df.head())


   cmc_rank      id slug         name           timestamp  ref_cur_id  \
0       164  1896.0   0x  0x Protocol 2017-08-16 23:59:59      2781.0   
1       164  1896.0   0x  0x Protocol 2017-08-17 23:59:59      2781.0   
2       164  1896.0   0x  0x Protocol 2017-08-18 23:59:59      2781.0   
3       164  1896.0   0x  0x Protocol 2017-08-19 23:59:59      2781.0   
4       164  1896.0   0x  0x Protocol 2017-08-20 23:59:59      2781.0   

  ref_cur_name   time_open           time_close            time_high  ...  \
0          USD  2017-08-16  2017-08-16 23:59:59  2017-08-16 17:29:26  ...   
1          USD  2017-08-17  2017-08-17 23:59:59  2017-08-17 01:59:26  ...   
2          USD  2017-08-18  2017-08-18 23:59:59  2017-08-18 10:29:34  ...   
3          USD  2017-08-19  2017-08-19 23:59:59  2017-08-19 14:30:13  ...   
4          USD  2017-08-20  2017-08-20 23:59:59  2017-08-20 00:15:04  ...   

       open      high       low     close      volume   market_cap  m_pct_1d  \
0  0.111725  0.280

In [9]:
# Calculate daily volume percentage (VolD%)
df['d_pct_vol_1d'] = df.groupby('slug')['volume'].pct_change()

'''# Calculate weekly volume percentage (VolW%)
df['VolW%'] = df.groupby('slug')['volume'].rolling(window=7).apply(lambda x: x.pct_change().mean() * 100).reset_index(level=0, drop=True)

# Calculate monthly volume percentage (VolM%)
df['VolM%'] = df.groupby('slug')['volume'].rolling(window=30).apply(lambda x: x.pct_change().mean() * 100).reset_index(level=0, drop=True)
'''


"# Calculate weekly volume percentage (VolW%)\ndf['VolW%'] = df.groupby('slug')['volume'].rolling(window=7).apply(lambda x: x.pct_change().mean() * 100).reset_index(level=0, drop=True)\n\n# Calculate monthly volume percentage (VolM%)\ndf['VolM%'] = df.groupby('slug')['volume'].rolling(window=30).apply(lambda x: x.pct_change().mean() * 100).reset_index(level=0, drop=True)\n"

In [20]:
import numpy as np
# Calculate the latest weekly volume percentage (VolW%)
# Ensure the DataFrame is sorted by 'timestamp' for each 'slug'
# Sort by 'timestamp' in descending order
df_sorted = df.sort_values(by='timestamp', ascending=True)


# Calculate latest weekly volume percentage
def latest_weekly_vol_percentage(group):
    if len(group) < 7:
        return np.nan
    latest_week = group.tail(7)
    return latest_week['volume'].pct_change().iloc[-1] * 100

df['d_pct_vol_1w'] = df.groupby('slug').apply(lambda x: latest_weekly_vol_percentage(x)).reset_index(level=0, drop=True)

# Calculate the latest monthly volume percentage (VolM%)
def latest_monthly_vol_percentage(group):
    if len(group) < 30:
        return np.nan
    latest_month = group.tail(30)
    return latest_month['volume'].pct_change().iloc[-1] * 100

df['d_pct_vol_1m'] = df.groupby('slug').apply(lambda x: latest_monthly_vol_percentage(x)).reset_index(level=0, drop=True)

print(df.head())




   cmc_rank      id slug         name           timestamp  ref_cur_id  \
0       164  1896.0   0x  0x Protocol 2017-08-16 23:59:59      2781.0   
1       164  1896.0   0x  0x Protocol 2017-08-17 23:59:59      2781.0   
2       164  1896.0   0x  0x Protocol 2017-08-18 23:59:59      2781.0   
3       164  1896.0   0x  0x Protocol 2017-08-19 23:59:59      2781.0   
4       164  1896.0   0x  0x Protocol 2017-08-20 23:59:59      2781.0   

  ref_cur_name   time_open           time_close            time_high  ...  \
0          USD  2017-08-16  2017-08-16 23:59:59  2017-08-16 17:29:26  ...   
1          USD  2017-08-17  2017-08-17 23:59:59  2017-08-17 01:59:26  ...   
2          USD  2017-08-18  2017-08-18 23:59:59  2017-08-18 10:29:34  ...   
3          USD  2017-08-19  2017-08-19 23:59:59  2017-08-19 14:30:13  ...   
4          USD  2017-08-20  2017-08-20 23:59:59  2017-08-20 00:15:04  ...   

      close      volume   market_cap  m_pct_1d  d_pct_cum_ret  d_pct_var  \
0  0.224399   5232600.

In [22]:
# Export DataFrame to CSV
df.to_csv('df_temp.csv', index=False)

print("DataFrame has been exported to 'cryptocurrency_volumes.csv'.")


DataFrame has been exported to 'cryptocurrency_volumes.csv'.


In [21]:
df_sorted.head()

,cmc_rank,id,slug,name,timestamp,ref_cur_id,ref_cur_name,time_open,time_close,time_high,...,close,volume,market_cap,m_pct_1d,d_pct_cum_ret,d_pct_var,d_pct_cvar,d_pct_vol_1d,d_pct_vol_1w,d_pct_vol_1m
143231,1,1.0,bitcoin,Bitcoin,2013-04-28 23:59:59,2781.0,USD,2013-04-28,2013-04-28 23:59:59,2013-04-28 00:45:00,...,127.954255,25956.84,1.419008e+09,NaN,NaN,-0.057924,-0.092178,NaN,NaN,NaN
771749,938,5.0,peercoin,Peercoin,2013-04-28 23:59:59,2781.0,USD,2013-04-28,2013-04-28 23:59:59,2013-04-28 19:40:02,...,0.386525,0.00,7.250189e+06,NaN,NaN,-0.089424,-0.143222,NaN,NaN,NaN
580087,18,2.0,litecoin,Litecoin,2013-04-28 23:59:59,2781.0,USD,2013-04-28,2013-04-28 23:59:59,2013-04-28 23:30:02,...,4.348400,0.00,7.463694e+07,NaN,NaN,-0.077167,-0.121012,NaN,NaN,NaN
771750,938,5.0,peercoin,Peercoin,2013-04-29 23:59:59,2781.0,USD,2013-04-29,2013-04-29 23:59:59,2013-04-29 13:15:03,...,0.408861,0.00,7.673915e+06,0.057787,0.057787,-0.089424,-0.143222,NaN,NaN,NaN
580088,18,2.0,litecoin,Litecoin,2013-04-29 23:59:59,2781.0,USD,2013-04-29,2013-04-29 23:59:59,2013-04-29 13:15:01,...,4.383900,0.00,7.538896e+07,0.008164,0.008164,-0.077167,-0.121012,NaN,NaN,NaN
